## Data Ingestion - Spark Streaming (Bronze Layer)

In [1]:
## Data Ingestion using PySpark Delta Lake
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StructType
from pyspark.sql.streaming import StreamingQuery

from pathlib import Path

In [2]:
# Initialize Spark Session
spark = (
    SparkSession.builder
    .appName('iot_data_ingestion')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
25/06/09 08:42:23 WARN Utils: Your hostname, HP-Pavilion-15-ec2121nr resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/06/09 08:42:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ubuntu/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-00c8f41c-631e-41ff-be68-5d485bf77962;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 215ms :: artifacts dl 9ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.2.0 from central in [default]
	io.delta#delta-storage;3.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0 

In [3]:
def infer_schema(file_path: str, format: str) -> StructType:
    
    match format:
        case 'json':
            inferred_schema = spark.read.json(file_path).schema
        case 'parquet':
            inferred_schema = spark.read.parquet(file_path).schema
        case _:
            raise ValueError(f"Unsupported format: {format}")
    
    return inferred_schema

In [ ]:
def ingest_data(
    path_to_file: str,  
    format: str,
    checkpoint_path: str,
    output_path: str
    ) -> StreamingQuery|None:
    
    try:
        # read streaming data with the specified schema
        stream_df = (
            spark.readStream
            .format(format)
            .option('maxFilesPerTrigger', 1)
            .schema(infer_schema(path_to_file, format))
            .load(path_to_file)
        )
        
        # write the stream to a Delta table
        return (
            stream_df.writeStream
            .format('delta')
            .outputMode('append')
            .option('checkpointLocation', checkpoint_path)
            .option('mergeSchema', 'true')
            .trigger(availableNow=True) # should be removed for production use/real-time streaming
            .start(output_path)
        )
        
    except Exception as e:
        print(f"Error reading stream from {path_to_file}: {e}")
        return

In [5]:
# ingest turbine data
turbine_query = ingest_data(
    path_to_file='../data/landing/turbine',
    format='json',
    checkpoint_path='../checkpoints/bronze_turbine',
    output_path='../data/bronze/bronze_turbine'
)
turbine_query.awaitTermination()

25/06/09 08:46:33 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/06/09 08:46:37 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
# ingest parts data
parts_stream = ingest_data(
    path_to_file='../data/landing/parts',
    format='json',
    checkpoint_path=f'../checkpoints/bronze_parts',
    output_path='../data/bronze/bronze_parts'
)
parts_stream.awaitTermination()

25/06/09 08:46:48 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [7]:
# ingest historical turbine status data
turbine_status_query = ingest_data(
    path_to_file='../data/landing/historical_turbine_status',
    format='json',
    checkpoint_path='../checkpoints/bronze_turbine_status',
    output_path='../data/bronze/bronze_turbine_status'
)
turbine_status_query.awaitTermination()

25/06/09 08:46:59 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [8]:
# ingest incoming turbine data
incoming_turbine_status_query = ingest_data(
    path_to_file='../data/landing/incoming_data',
    format='parquet',
    checkpoint_path='../checkpoints/bronze_incoming_data',
    output_path='../data/bronze/bronze_incoming_data'
)
incoming_turbine_status_query.awaitTermination()

25/06/09 08:47:06 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
